<a href="https://colab.research.google.com/github/MK316/Spring2024/blob/main/Engpro/Engpro24_Mid2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install gradio matplotlib

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
df = pd.read_csv('/content/drive/My Drive/EngproS24/Engpro24MT.csv', encoding="utf-8-sig")
df.head()

In [ ]:
print(df.columns)

In [ ]:
# Pivot the table
pivot_df = df.pivot_table(index='Email', columns='Test', values='Score')
print(pivot_df.head())


In [ ]:
# Score Mean (1st vs. 2nd)
df.groupby('Test')['Score'].mean()

Boxplots: 1st vs. 2nd

In [ ]:
import matplotlib.pyplot as plt

# Creating a boxplot of scores grouped by the 'Test' category
df.boxplot(by='Test', column='Score', grid=True)

# Adding titles and labels
plt.title('Boxplot of Scores by Test Type')
plt.suptitle('')  # Suppress the default title to clean up the plot
plt.xlabel('Test')
plt.ylabel('Score')

# Show the plot
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# Step 1: Select rows by 'Test' and create two separate dataframes
df1 = df[df['Test'] == '1st']
df2 = df[df['Test'] == '2nd']

# Step 2: Sort df1 by 'Score' in decreasing order
df1_sorted = df1.sort_values(by='Score', ascending=False)

# Step 3: Get the name order from the sorted df1
name_order = df1_sorted['Name'].tolist()

# Step 4: Reorder df2 to match the name order of df1
df2_ordered = df2.set_index('Name').loc[name_order].reset_index()

# Plotting
plt.figure(figsize=(10, 6))

# Step 5: Plot df1 scores
plt.scatter(df1_sorted['Name'], df1_sorted['Score'], color='blue', label='1st Test')

# Step 6: Plot df2 scores using the same order
plt.scatter(df2_ordered['Name'], df2_ordered['Score'], color='red', label='2nd Test')

# Formatting the plot
plt.xlabel('Name')
plt.ylabel('Score')
plt.title('Scores by Name and Test')
plt.ylim(0,210)
plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability
plt.grid(True)
plt.legend()

plt.show()


# Score to check

In [ ]:
# Create a pivot table to organize scores
pivot_df = df.pivot(index='Email', columns='Test', values='Score').reset_index()
pivot_df.columns.name = None  # remove the categorization name

# Merge this pivot table with the main df to add SID and Name (assuming SID and Name are constant per Email)
info_df = df[['Email', 'SID', 'Name']].drop_duplicates()
final_df = pd.merge(info_df, pivot_df, on='Email')

import gradio as gr

def check_scores(email):
    if email in final_df['Email'].values:
        user_data = final_df[final_df['Email'] == email].iloc[0]
        return str(user_data['SID']), user_data['Name'], str(user_data['1st']), str(user_data['2nd'])
    else:
        return "No SID found", "No name found", "No score for 1st Test", "No score for 2nd Test"

iface = gr.Interface(
    fn=check_scores,
    inputs=gr.Textbox(label="Enter Your Email"),
    outputs=[
        gr.Textbox(label="SID"),
        gr.Textbox(label="Name"),
        gr.Textbox(label="1st Test Score"),
        gr.Textbox(label="2nd Test Score")
    ],
    title="Check Your Test Scores",
    description="Enter your email to retrieve your SID, Name, and Test Scores for the 1st and 2nd tests."
)

iface.launch(debug=True)
